In [22]:
import os
import time
import subprocess
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Configurações do projeto
REPO_PATH = "anything-llm"
OUTPUT_FILE = "padroes_arquiteturais.txt"
ARQUIVOS_ANALISADOS_FILE = "arquivos_analisados.txt"  # NOVO: arquivo da entrada

# Padrões arquiteturais para busca
ARCHITECTURAL_PATTERNS = [
    "MVC - Model-View-Controller separation pattern",
    "Microservices - Independently deployable services architecture",
    "Client-Server - Centralized server with multiple clients",
    "Layered Architecture - Hierarchical layer separation",
    "Event-Driven - Asynchronous event-based communication",
    "Service-Oriented - Reusable services with standardized interfaces",
    "Component-Based - Reusable component architecture",
    "Repository Pattern - Data access abstraction layer",
    "Factory Pattern - Object creation pattern",
    "Singleton Pattern - Single instance global access"
]

# Carregar modelo de embeddings
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def get_recent_commits(num_commits=100):
    """Obtém os últimos commits do repositório"""
    try:
        result = subprocess.run([
            'git', 'log', f'-{num_commits}', '--oneline', '--format=%H %s'
        ], capture_output=True, text=True, cwd=REPO_PATH)

        if result.returncode != 0:
            print("Erro ao obter commits")
            return []

        commits = []
        lines = result.stdout.strip().split('\n')
        for line in lines:
            if line.strip():
                parts = line.split(' ', 1)
                if len(parts) == 2:
                    commit_hash, message = parts
                    commits.append({
                        'hash': commit_hash,
                        'message': message,
                        'type': 'commit'
                    })

        return commits

    except Exception as e:
        print(f"Erro ao obter commits: {e}")
        return []

def get_commit_details(commit_hash):
    """Obtém detalhes de um commit específico"""
    try:
        result = subprocess.run([
            'git', 'show', '--stat', '--oneline', commit_hash
        ], capture_output=True, text=True, cwd=REPO_PATH)

        if result.returncode == 0:
            return result.stdout
        return ""

    except Exception as e:
        print(f"Erro ao obter detalhes do commit {commit_hash}: {e}")
        return ""

def find_text_files():
    """Encontra arquivos de texto, README e documentação no repositório"""
    text_files = []

    for root, dirs, files in os.walk(REPO_PATH):
        if 'node_modules' in dirs:
            dirs.remove('node_modules')
        if '.git' in dirs:
            dirs.remove('.git')

        for file in files:
            if file.lower().endswith(('.md', '.txt', '.rst')) or file.lower() in ['readme', 'documentation']:
                full_path = os.path.join(root, file)
                text_files.append({
                    'path': full_path,
                    'type': 'file'
                })

    return text_files[:15]

def save_arquivos_analisados(commits, text_files):  # NOVA FUNÇÃO
    """Salva a lista de arquivos que serão analisados em um txt"""
    with open(ARQUIVOS_ANALISADOS_FILE, 'w', encoding='utf-8') as f:
        f.write("# Arquivos Selecionados para Análise\n\n")

        f.write("## Commits Analisados\n")
        f.write(f"Total: {len(commits)} commits\n\n")
        for i, commit in enumerate(commits[:10], 1):  # Mostra primeiros 10
            f.write(f"{i}. {commit['hash'][:8]} - {commit['message']}\n")
        if len(commits) > 10:
            f.write(f"... e mais {len(commits) - 10} commits\n")

        f.write("\n## Arquivos de Texto Analisados\n")
        f.write(f"Total: {len(text_files)} arquivos\n\n")
        for i, file_info in enumerate(text_files, 1):
            relative_path = file_info['path'].replace(REPO_PATH + '/', '')
            f.write(f"{i}. {relative_path}\n")

def analyze_content(content, pattern_embeddings):
    """Analisa o conteúdo em busca de padrões arquiteturais"""
    if not content.strip():
        return []

    # Gerar embedding do conteúdo
    content_embedding = model.encode(content)

    # Calcular similaridade com os padrões arquiteturais
    similarities = cosine_similarity([content_embedding], pattern_embeddings)[0]

    # Identificar padrões com probabilidade acima de 25%
    detected_patterns = []
    for i, similarity in enumerate(similarities):
        if similarity > 0.25:
            detected_patterns.append({
                'pattern': ARCHITECTURAL_PATTERNS[i],
                'probability': float(similarity)
            })

    # Ordenar por probabilidade decrescente
    detected_patterns.sort(key=lambda x: x['probability'], reverse=True)
    return detected_patterns

def main():
    # Iniciar contagem de tempo
    start_time = time.perf_counter()

    print("Iniciando analise de padroes arquiteturais...")
    print("Prompt: Considere que o seu objetivo é encontrar padrões arquiteturais no repositório especificado. APENAS utilizando os arquivos e pastas especificados, faça uma busca no repositório, cite quais são os possíveis modelos encontrados, a probabilidade de ser cada um e retorne com os resultados em um arquivo .txt")
    print("Analisando ultimos 100 commits e arquivos de texto...")

    # Preparar embeddings dos padrões arquiteturais
    pattern_embeddings = model.encode(ARCHITECTURAL_PATTERNS)

    # Obter dados para análise
    commits = get_recent_commits(100)
    text_files = find_text_files()

    print(f"Encontrados {len(commits)} commits e {len(text_files)} arquivos de texto")

    # NOVO: Salvar lista de arquivos que serão analisados
    save_arquivos_analisados(commits, text_files)
    print(f"Lista de arquivos salva em: {ARQUIVOS_ANALISADOS_FILE}")

    # Analisar commits e arquivos
    results = {}

    # Processar commits
    for commit in commits:
        commit_hash = commit['hash']
        commit_message = commit['message']

        commit_details = get_commit_details(commit_hash)
        commit_content = f"{commit_message} {commit_details}"[:1500]

        patterns = analyze_content(commit_content, pattern_embeddings)
        if patterns:
            source_key = f"COMMIT: {commit_hash[:8]} - {commit_message}"
            results[source_key] = patterns

    # Processar arquivos de texto
    for file_info in text_files:
        file_path = file_info['path']
        file_name = os.path.basename(file_path)

        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read(2000)

            patterns = analyze_content(content, pattern_embeddings)
            if patterns:
                source_key = f"FILE: {file_path.replace(REPO_PATH + '/', '')}"
                results[source_key] = patterns

        except Exception as e:
            print(f"Erro ao analisar arquivo {file_name}: {e}")

    # Gerar relatório em arquivo txt
    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        f.write("# Padroes Arquiteturais de Software Identificados\n\n")

        # Agrupar resultados por padrão arquitetural
        grouped_patterns = {}
        for source, patterns in results.items():
            for pattern_info in patterns:
                pattern_name = pattern_info['pattern']
                if pattern_name not in grouped_patterns:
                    grouped_patterns[pattern_name] = []
                grouped_patterns[pattern_name].append({
                    'source': source,
                    'probability': pattern_info['probability']
                })

        # Escrever padrões identificados
        if grouped_patterns:
            f.write("## Padroes Identificados\n\n")
            for pattern, occurrences in grouped_patterns.items():
                avg_probability = np.mean([occ['probability'] for occ in occurrences])
                pattern_short = pattern.split(' - ')[0]
                f.write(f"### {pattern_short}\n")
                f.write(f"Probabilidade Media: {avg_probability:.1%}\n")
                f.write("Fontes relacionadas:\n")

                for occ in sorted(occurrences, key=lambda x: x['probability'], reverse=True)[:3]:
                    source_type = "COMMIT" if occ['source'].startswith('COMMIT') else "FILE"
                    f.write(f"- [{source_type}] {occ['source']} ({occ['probability']:.1%})\n")
                f.write("\n")
        else:
            f.write("Nenhum padrao arquitetural significativo foi identificado.\n\n")

        f.write("# Resumo\n\n")
        f.write(f"Repositorio analisado: {REPO_PATH}\n")
        f.write(f"Commits analisados: {len(commits)}\n")
        f.write(f"Arquivos de texto analisados: {len(text_files)}\n")
        f.write(f"Padroes identificados: {len(grouped_patterns)}\n")
        f.write("Metodologia: Analise semantica de commits e documentacao utilizando embeddings de texto\n")
        f.write("Modelo utilizado: sentence-transformers/all-MiniLM-L6-v2\n")

    # Calcular tempo de execução
    execution_time = time.perf_counter() - start_time

    print(f"Analise concluida!")
    print(f"Relatorio salvo em: {OUTPUT_FILE}")
    print(f"Lista de entrada salva em: {ARQUIVOS_ANALISADOS_FILE}")
    print(f"Tempo de execucao: {execution_time:.2f} segundos")

# Executar a análise
if __name__ == "__main__":
    main()

Iniciando analise de padroes arquiteturais...
Prompt: Considere que o seu objetivo é encontrar padrões arquiteturais no repositório especificado. APENAS utilizando os arquivos e pastas especificados, faça uma busca no repositório, cite quais são os possíveis modelos encontrados, a probabilidade de ser cada um e retorne com os resultados em um arquivo .txt
Analisando ultimos 100 commits e arquivos de texto...
Encontrados 100 commits e 15 arquivos de texto
Lista de arquivos salva em: arquivos_analisados.txt
Analise concluida!
Relatorio salvo em: padroes_arquiteturais.txt
Lista de entrada salva em: arquivos_analisados.txt
Tempo de execucao: 12.34 segundos


In [23]:
# Célula para visualizar os arquivos gerados
from google.colab import files
import os

# Verificar se os arquivos foram criados
arquivos_gerados = [OUTPUT_FILE, ARQUIVOS_ANALISADOS_FILE]

for arquivo in arquivos_gerados:
    if os.path.exists(arquivo):
        print(f"=== CONTEÚDO DE {arquivo} ===")
        print(f"Tamanho: {os.path.getsize(arquivo)} bytes")
        print("\n" + "="*50)

        with open(arquivo, 'r', encoding='utf-8') as f:
            content = f.read()
            print(content)

        print("\n" + "="*50)
        print()
    else:
        print(f"Arquivo {arquivo} não encontrado.")
        print()

print("Para baixar os arquivos, execute a próxima célula ou:")
print("1. Clique no ícone de pasta no menu lateral")
print("2. Encontre os arquivos .txt")
print("3. Clique com botão direito → Download")

=== CONTEÚDO DE padroes_arquiteturais.txt ===
Tamanho: 1296 bytes

# Padroes Arquiteturais de Software Identificados

## Padroes Identificados

### Microservices
Probabilidade Media: 31.9%
Fontes relacionadas:
- [FILE] FILE: server/utils/vectorDbProviders/milvus/MILVUS_SETUP.md (38.3%)
- [COMMIT] COMMIT: 599a3fd8 - Microsoft Foundry Local LLM provider & agent provider (#4435) (34.9%)
- [COMMIT] COMMIT: 71cd46ce - 1.9.0 tag (27.7%)

### Component-Based
Probabilidade Media: 25.8%
Fontes relacionadas:
- [COMMIT] COMMIT: c2e7ccc0 - Reimplement Cohere models for basic chat (#4489) (25.8%)

### Event-Driven
Probabilidade Media: 29.3%
Fontes relacionadas:
- [COMMIT] COMMIT: d6f0d305 - Enable real-time agent tool call streaming for all providers (#4279) (34.6%)
- [COMMIT] COMMIT: 5922349b - feat: Implement CometAPI integration for chat completions and model m… (#4379) (32.1%)
- [COMMIT] COMMIT: 8cdadd8c - Sync models from remote for FireworksAI (#4475) (25.3%)

### Repository Pattern
Probabili

In [25]:
# Célula para baixar os arquivos
from google.colab import files

arquivos_para_download = [OUTPUT_FILE, ARQUIVOS_ANALISADOS_FILE]

for arquivo in arquivos_para_download:
    if os.path.exists(arquivo):
        files.download(arquivo)
        print(f"✅ {arquivo} - Download iniciado!")
    else:
        print(f"❌ {arquivo} - Arquivo não encontrado")

print("Todos os downloads foram iniciados!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ padroes_arquiteturais.txt - Download iniciado!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ arquivos_analisados.txt - Download iniciado!
Todos os downloads foram iniciados!
